# Otic Mesenchyme and Sensory Epithelium Cell-Cell Communication Analysis

Prepared by: Abel P. David (Northwestern University)

Date: 11/6/2025

Prepared for: Ippei Kishimoto and Alan Cheng (Stanford University)

In [ ]:
library(CellChat)
library(patchwork)
library(NMF)
library(circlize)
library(ComplexHeatmap)
library(Seurat)
library(Matrix)
library(magrittr)
library(tidyverse)
library(SingleCellExperiment)
library(remotes)
library(renv)
library(future)
library(furrr)
library(SeuratDisk)
library(devtools)
library(Seurat)
library(SeuratDisk)
library(harmony)

# E15/E16 Combination

In [ ]:
data_name = "E15E16_PSC_Separated"
so_e15 <- readRDS("rose_2023_e15.rds")
so_e16 <- readRDS("kolla_2020_e16.rds")
DimPlot(so_e15, group.by = "cluster_label", label = TRUE)

In [ ]:
so_e15@meta.data <- so_e15@meta.data %>%
  mutate(
    selection = case_when(
      cluster_label  %in% c("Pro Mes", "TypeI Mes", "TypeIE Mes", "TypeII Mes", "TypeIII Mes", "TypeIV Mes") ~ "Yes",
      TRUE ~ "No"
    )
)

In [ ]:
DimPlot(so_e15, group.by = "selection")

In [ ]:
so_e15$cluster_combined <- so_e15$cluster_label
so_e15 <- subset(so_e15, subset = cluster_label %in% c("Pro Mes", "TypeI Mes", "TypeIE Mes", "TypeII Mes", "TypeIII Mes", "TypeIV Mes"))
DimPlot(so_e15, group.by = "cluster_label")

In [ ]:
DimPlot(so_e16, group.by = "cell_type")
so_e16$cluster_combined <- so_e16$cell_type

In [ ]:
so_e15$batch <- "E15"
so_e16$batch <- "E16"

In [ ]:
so <- merge(so_e15, y = so_e16, merge.data = TRUE)
so[["RNA"]] <- JoinLayers(so[["RNA"]])

In [ ]:
so <- NormalizeData(so)
so <- FindVariableFeatures(so, nfeatures = 3000)
so <- ScaleData(so, verbose = FALSE)
so <- RunPCA(so, npcs = 30, verbose = FALSE)

In [ ]:
so <- RunHarmony(object = so, group.by.vars = "batch", reduction.use = "pca", plot_convergence = TRUE)

In [ ]:
so <- RunUMAP(so, reduction = "harmony", dims = 1:30)
so <- FindNeighbors(so, reduction = "harmony", dims = 1:30)
so <- FindClusters(so, resolution = 0.5)

In [ ]:
DimPlot(so, group.by = "cluster_combined", label = TRUE)

In [ ]:
DimPlot(so, label = TRUE)

In [ ]:
so@meta.data <- so@meta.data %>%
  mutate(
    final_cluster = case_when(
      seurat_clusters %in% c("5") ~ "HC",
      seurat_clusters %in% c("8", "13") ~ "PsC",
      seurat_clusters %in% c("2", "3", "4", "6", "7") ~ "POM",
      TRUE ~ "Epithelial"
    )
)

In [ ]:
so@meta.data <- so@meta.data %>%
  mutate(
    sub_cluster = case_when(
      seurat_clusters %in% c("5") ~ "HC",
      seurat_clusters %in% c("8", "13") ~ "PsC",
      seurat_clusters %in% c("2", "3", "4", "6", "7") ~ "POM",
      seurat_clusters %in% c("0", "1", "9", "14") ~ "GER",
      seurat_clusters %in% c("10", "12") ~ "LER",
      seurat_clusters %in% c("11") ~ "Roof",
      TRUE ~ "Epithelial"
    )
)

In [ ]:
DimPlot(so, group.by = "sub_cluster", label = TRUE)

In [ ]:
pdf(paste0(data_name, "_Sub_Cluster_UMAP.pdf"), width = 6, height = 5, family = "Helvetica")
DimPlot(so, group.by = "sub_cluster")
dev.off()

In [ ]:
DimPlot(so, group.by = "final_cluster", label = TRUE)

In [ ]:
saveRDS(so, file = "e15_e16_harmony_separated.rds")